In [4]:
import psycopg2
import psycopg2.extras
import os



def conecta_db():
      con = psycopg2.connect(host='localhost', 
                         database='postgres',
                         user='postgres', 
                         password='postgres')
      return con


def criar_db(sql):
    con = conecta_db()
    cur = con.cursor()
    cur.execute(sql)
    con.commit()
    con.close()


def inserir_db(sql):
    con = conecta_db()
    cur = con.cursor()
    try:
        cur.execute(sql)
        con.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        con.rollback()
        cur.close()
        return 1
    cur.close()


In [5]:

import numpy as np

# centro_replace = {
#         'C. O. ALG - ALGARVE': '8811455',
#         'C. O. AVEIRO': '8811417',
#         'C. O. BRAGA': '8817018',
#         'C. O. COIMBRA': '8818171',
#         'C. O. LEIRIA': '8811456',
#         'C. O. MEM MARTINS': '8811373',
#         'C. O. PERAFITA': '8818154',
#         'C. O. VISEU': '8811457',
#         'C.O. LOURES - MARL': '8811496',
#         'C.O. PALMELA': '8819009',
#         'CO DEVESAS': '8819001',
#         'CO PINHEIRO DE FORA': '8819002',
#     }

#selecionar o centro e as datas:
    
CO='8818171'

data_validation = []
con = conecta_db()
cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)

cur.execute("""
    SELECT * FROM expresso_2023 
    WHERE centro = %s 
    AND TO_DATE(data_criacao, 'DD/MM/YYYY') >= TO_DATE('01/11/2023', 'DD/MM/YYYY') 
    AND TO_DATE(data_criacao, 'DD/MM/YYYY') <= TO_DATE('30/11/2023', 'DD/MM/YYYY')
""", (CO,))


for record in cur.fetchall():
    if record[0] not in data_validation:
        data_validation.append(record[0])
cur.close()

data_validation=np.array(data_validation)

data_validation

array(['16/11/2023', '15/11/2023', '17/11/2023', '18/11/2023',
       '20/11/2023', '21/11/2023', '22/11/2023', '24/11/2023',
       '23/11/2023', '27/11/2023', '25/11/2023', '29/11/2023',
       '28/11/2023', '30/11/2023', '13/11/2023', '14/11/2023'],
      dtype='<U10')

In [6]:
#join

import numpy as np
import pandas as pd
import os
import re

for data in data_validation:
    
    con = conecta_db()
    cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
    query = f"SELECT * FROM banca_2023_fn WHERE TO_DATE(data_criacao, 'DD/MM/YYYY') = TO_DATE('{data}', 'DD/MM/YYYY') and data_evento IS NOT NULL;"
    cur.execute(query)
    column_names = [desc[0] for desc in cur.description]
    result_data = cur.fetchall()
    cur.close()
    con.close()
    
    df_banca = pd.DataFrame(result_data, columns=column_names)

    df_banca['giro'] = df_banca['giro'].str.strip().str[:5]

    lis = ['8811455', '8811417', '8817018', '8818171', '8811456',
        '8811373', '8818154', '8811457', '8811496', '8819009',
        '8819001', '8819002']

    df_banca = df_banca[df_banca['centro'].isin(lis)]

    df_banca = df_banca[df_banca['giro'].str.contains(r'^\d{4}[A-Za-z]$')]
    

    df_banca['data_evento'] = pd.to_datetime(df_banca['data_evento'], format='%d/%m/%Y %H:%M', errors='coerce')

    df_banca['data_evento'] = df_banca['data_evento'].dt.strftime('%H:%M')

    df_banca['data_evento'] = df_banca['data_evento'].astype(str)
    
    df_banca.dropna(subset=['data_evento'], inplace=True)
    df_banca.dropna(subset=['janela_horaria'], inplace=True)
    

    #recolhas
        
    con = conecta_db()
    cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
    query_r = f"SELECT * FROM recolhas_2023_fn WHERE TO_DATE(data_criacao, 'DD/MM/YYYY') = TO_DATE('{data}', 'DD/MM/YYYY') and cod_t_even ='Abatido' and data_evento IS NOT NULL;"
    cur.execute(query_r)
    column_names = [desc[0] for desc in cur.description]
    result_data_rec = cur.fetchall()
    cur.close()
    con.close()

    df_recolhas = pd.DataFrame(result_data_rec, columns=column_names)


    df_recolhas['giro'] = df_recolhas['giro'].str.strip().str[:5]

    centro_replace = {
        'C. O. ALG - ALGARVE': '8811455',
        'C. O. AVEIRO': '8811417',
        'C. O. BRAGA': '8817018',
        'C. O. COIMBRA': '8818171',
        'C. O. LEIRIA': '8811456',
        'C. O. MEM MARTINS': '8811373',
        'C. O. PERAFITA': '8818154',
        'C. O. VISEU': '8811457',
        'C.O. LOURES - MARL': '8811496',
        'C.O. PALMELA': '8819009',
        'CO DEVESAS': '8819001',
        'CO PINHEIRO DE FORA': '8819002',
    }

    df_recolhas['centro'] = df_recolhas['centro'].replace(centro_replace)


    lis = ['8811455', '8811417', '8817018', '8818171', '8811456',
        '8811373', '8818154', '8811457', '8811496', '8819009',
        '8819001', '8819002']

    df_recolhas = df_recolhas[df_recolhas['centro'].isin(lis)]

    df_recolhas = df_recolhas[df_recolhas['giro'].str.contains(r'^\d{4}[A-Za-z]$')]

    df_recolhas['data_evento'] = df_recolhas['data_evento'].astype(str)
    
    df_recolhas = df_recolhas[df_recolhas['data_evento'] != '']
    
    df_recolhas.dropna(subset=['data_evento'], inplace=True)
    

    # express0
    
    con = conecta_db()
    cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
    query_e= f"SELECT * FROM expresso_2023 WHERE TO_DATE(data_criacao, 'DD/MM/YYYY') = TO_DATE('{data}', 'DD/MM/YYYY')"
    cur.execute(query_e)
    column_names = [desc[0] for desc in cur.description]
    result_data_e = cur.fetchall()
    cur.close()
    con.close()

    df_expresso = pd.DataFrame(result_data_e, columns=column_names)

    df_expresso['giro'] = df_expresso['giro'].str.strip().str[:5]
    df_expresso['cp'] = df_expresso['cp'].str.strip()
    

    df_expresso['data_evento'] = pd.to_datetime(df_expresso['data_evento'], format='%d/%m/%Y %H:%M:%S', errors='coerce')

    df_expresso['data_evento'] = df_expresso['data_evento'].dt.strftime('%H:%M')

    df_expresso['data_evento'] = df_expresso['data_evento'].astype(str)
    
    df_expresso.dropna(subset=['data_evento'], inplace=True)
    
    df_expresso["giro"] = df_expresso["giro"].replace("2900 ", "2900C")

    #"SIG_POSTAL"
    
    con = conecta_db()
    cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
    query = "select Cp7, longitude_x, latitude_y from centroid_cp7_sig"
    cur.execute(query)
    SIG_POSTAL_records = cur.fetchall()
    cur.close()
    con.close()

    # Convert the list of dictionaries to a DataFrame
    
    con = conecta_db()
    cur = con.cursor(cursor_factory=psycopg2.extras.DictCursor)
    query = "select Cp7,longitude_x,latitude_y  from centroid_cp7_sig"
    cur.execute(query)
    SIG_POSTAL = cur.fetchall()
    
    SIG_POSTAL=pd.DataFrame(SIG_POSTAL,columns=['cp7','longitude(X)','latitude(Y)'])
    SIG_POSTAL['cp7']= SIG_POSTAL['cp7'].str.strip()
    
    merged_df = df_expresso.merge(SIG_POSTAL[['cp7', 'longitude(X)', 'latitude(Y)']], left_on='cp', right_on='cp7', how='left')
    merged_df
    # Update the 'latitude' and 'longitude' columns in df_expresso with non-null values from the second DataFrame
    df_expresso.loc[df_expresso['latitude'].isnull(), 'latitude'] = merged_df.loc[df_expresso['latitude'].isnull(), 'latitude(Y)']
    df_expresso.loc[df_expresso['longitude'].isnull(), 'longitude'] = merged_df.loc[df_expresso['longitude'].isnull(), 'longitude(X)']
    
    # teste
    
    def remove_non_printable_chars(s):
        if isinstance(s, float):  # Check if the value is a float
            return str(s)  # Convert float to string
        return re.sub(r'[^\x00-\x7F]+', '', str(s))

    # Apply the function to the 'latitude' and 'longitude' columns
    df_expresso['latitude'] = df_expresso['latitude'].apply(remove_non_printable_chars)
    df_expresso['longitude'] = df_expresso['longitude'].apply(remove_non_printable_chars)

    # If you also need to remove a specific character, for example ''
    df_expresso['latitude'] = df_expresso['latitude'].str.replace('', '')
    df_expresso['longitude'] = df_expresso['longitude'].str.replace('', '')
    df_expresso = df_expresso[df_expresso['latitude'] != 'nan']
    df_expresso = df_expresso[df_expresso['longitude'] != 'nan']
    
    #

    df = pd.concat([df_expresso, df_banca, df_recolhas], ignore_index=True)
    df.columns = df.columns.str.upper()
    df=df[df['GIRO'].str[:1]==df['CP'].str[:1]]
    df=df[df.LATITUDE != '0']
    df = df[df['LATITUDE'] != '']
    df=df[df.LATITUDE != '0.0']
    df.dropna(subset=['LATITUDE'], inplace=True)
    df.dropna(subset=['LONGITUDE'], inplace=True)
    df['DATA_CRIACAO']=df['DATA_CRIACAO'].replace('/', '')
    
    df = df[~(df['LATITUDE'] == 0.0)]
    df = df[~(df['JANELA_HORARIA'] == '00:00')]
    df = df[~(df['JANELA_HORARIA'].str[:2] == '00')]
    df = df[~(df['JANELA_HORARIA'].str[:2] == '01')]
    df = df[~(df['JANELA_HORARIA'].str[:2] == '02')]
    df = df[~(df['JANELA_HORARIA'].str[:2] == '03')]
    df = df[~(df['JANELA_HORARIA'].str[:2] == '04')]
    df = df[~(df['JANELA_HORARIA'].str[:2] == '05')]
    df = df[~(df['JANELA_HORARIA'].str[:2] == '06')]
    df = df[~(df['JANELA_HORARIA'].str[:2] == '23')]


    df = df[df['CENTRO'] == CO].reset_index(drop=True)

    # Agrupar os dados pelo centro e pela data
    grupos_centro_data = df.groupby(['CENTRO', 'DATA_CRIACAO'])

    # Loop para cada centro e para cada dia
    for (centro, data), grupo in grupos_centro_data:
        
        # Assuming the DataFrame is named "df"
        df.sort_values(by=['GIRO', 'DATA_EVENTO'], inplace=True)

        # Create a list to store the final output
        output_list = []

        current_giro = None

        for _, row in df.iterrows():
            giro = row['GIRO']
            loptica = row['LOPTICA']

            if current_giro != giro:
                # Append a blank line before starting the new giro
                if current_giro is not None:
                    output_list.append('')

                # Append giro to the output_list when it changes
                output_list.append(giro)
                current_giro = giro

            # Append loptica to the output_list
            output_list.append(loptica)

        # Insert "Name" at the beginning of the output_list
        output_list.insert(0, 'Nome')

        # Convert the output_list to a DataFrame, skipping the first occurrence of "Name"
        df_seq_rota = pd.DataFrame({'Nome': output_list}).iloc[1:]

        # Now result_df contains the desired result
        #print(df_seq_rota)


        #csv_file_path = os.path.join(os.path.expanduser('~'), 'Downloads', 'seq2.csv')
        #df_seq_rota.to_csv(csv_file_path, index=False)
        unique_data_criacao = df['DATA_CRIACAO'].unique()
        date_str = unique_data_criacao[0].replace('/', '')

        file_name=str(CO)+"_"+str(date_str)
        


        df1 = df.loc[:, ['LOPTICA']]
        df1 = df1.rename(columns={'LOPTICA':'Nome'})

        # df1 = df.loc[:, ['LOPTICA']]
        # df1['Nome']= df1['LOPTICA'].str.cat(df.loc[:, ['COD_T_EVEN']],sep='_')
        # df1 = df1.drop('LOPTICA', axis=1)

        df2 = df.loc[:, ['MORADA']]
        df2 = df2.rename(columns={'MORADA':'Localização da entrega'})
        df2_na= df2.fillna('vazio')
        df3= df.loc[:, ['CP']]
        df3 = df3.rename(columns={'CP':'Código Postal de Entrega'})
        df4 = df.loc[:, ['LOCALIDADE']]
        df4 = df4.rename(columns={'LOCALIDADE':'Cidade de Entrega'})
        df5 ={'País de Entrega':['PORTUGAL']}
        df6= df.loc[:, ['LATITUDE']]
        df6 = df6.rename(columns={'LATITUDE':'Latitude da entrega'})
        df7 = df.loc[:, ['LONGITUDE']]
        df7 = df7.rename(columns={'LONGITUDE':'Longitude da entrega'})
        df8 = df.loc[:, ['JANELA_HORARIA']]
        df9 = df.loc[:, ['COD_T_EVEN']]
        df9 = df7.rename(columns={'COD_T_EVEN':'Grupo'})

        df12 =df.loc[:, ['GIRO']]
        df12['GIRO'] = df12['GIRO'].str[:5]
        df12_v =df.loc[:, ['GIRO']]
        df12_v['GIRO'] = df12['GIRO'].str[:4]
        df12_vn = pd.to_numeric(df12_v['GIRO'], errors='coerce').notnull()
        DF12_VN_F = df12_v[df12_vn]

        df13 =df.loc[:, ['DATA_EVENTO']]
        df13 = df13.assign(DATA_EVENTO=df13['DATA_EVENTO'].str[-8:])
        df13['Grupo'] = df13['DATA_EVENTO'].str.cat(df12['GIRO'],sep=',')
        df13 = df13.drop(columns=['DATA_EVENTO'])

        df13_vf =df.loc[:, ['DATA_EVENTO']]
        df13_vf = df13_vf.assign(DATA_EVENTO=df13_vf['DATA_EVENTO'].str[-8:])
        df13_vf['Grupo'] = df13_vf['DATA_EVENTO'].str.cat(df12['GIRO'],sep=',').str.cat(DF12_VN_F['GIRO'],sep=',')
        df13_vf = df13_vf.drop(columns=['DATA_EVENTO'])


        ##--extração das janelas horárias--##

        dfi=df['LOPTICA'].str.slice(0, 2)
        dfi2=df.loc[:, ['JANELA_HORARIA']]
        dfi3=pd.merge(dfi, dfi2, left_on='LOPTICA', right_on='JANELA_HORARIA', how='right')
        dfi3aux=dfi3['JANELA_HORARIA'].str[:2]+":00"

        dfifaux=dfi3['JANELA_HORARIA'].str[3:5]+":00"
        #print(dfifaux)


        ##--Inicio do Intervalo da Entrega--##
        dfiaux=np.where(dfi =="ED", "09:00",np.where(dfi =="EG", "09:00",np.where(dfi =="FA", "08:00",  np.where(dfi =="FC", "08:00",  np.where(dfi =="FD", "08:00",  
                        np.where(dfi =="ES", "08:00",  np.where(dfi =="EI", "10:00",  np.where(dfi =="EN", "08:00",  np.where(dfi =="EA", "08:00",  np.where(dfi =="EC", "08:00",  
                        np.where(dfi =="DA", "08:00",np.where(dfi =="EM", "08:00",np.where(dfi =="FQ", "08:00",np.where(dfi =="EX", "08:00",np.where(dfi =="ER", "08:00",np.where(dfi =="EO", "08:00",
                        np.where(dfi =="EO", "08:00",np.where(dfi =="EH", "08:00",np.where(dfi =="VO", "08:00",np.where(dfi =="DB", "08:00",np.where(dfi =="EQ", "08:00",np.where(dfi =="DW", "08:00",np.where(dfi =="PA", "08:00",np.where(dfi =="DT", "08:00",np.where(dfi =="TG", "08:00",
                        np.where(dfi =="RH", "08:00",np.where(dfi =="RC", "08:00",np.where(dfi =="RU", "08:00",np.where(dfi =="RG", "08:00",np.where(dfi =="RY", "08:00",np.where(dfi =="RF", "08:00",np.where(dfi =="RG", "08:00",
                        np.where(dfi =="AB", "08:00",np.where(dfi =="RL", "08:00",np.where(dfi =="OC", "08:00",np.where(dfi =="LH", "08:00",np.where(dfi =="RA", "08:00",np.where(dfi =="AD", "08:00",
                        np.where(dfi =="RE", "08:00",np.where(dfi =="RV", "08:00",np.where(dfi =="RM", "08:00",np.where(dfi =="CO", "08:00",np.where(dfi =="UG", "08:00",
                        np.where(dfi =="LS", "08:00",np.where(dfi =="LA", "08:00",np.where(dfi =="RR", "08:00",np.where(dfi =="RQ", "08:00",np.where(dfi =="LD", "08:00",
                        np.where(dfi =="QA", "08:00","08:00")))))))))))))))))))))))))))))))))))))))))))))))))                                                                            

        dfif=pd.DataFrame(dfiaux,columns=['Inicio do Intervalo da Entrega'])                                                                                  

        dfif=np.where(dfi3['JANELA_HORARIA'].str.len() >= 4, dfi3aux,dfiaux)
        #print(dfif)
        dfif=pd.DataFrame(dfif,columns=['Inicio do Intervalo da Entrega'])


        ##--fim do intervalo de entrega--##

        dfiaux2=np.where(dfi =="ED", "13:00",np.where(dfi =="EG", "19:00",np.where(dfi =="FA", "19:00",  np.where(dfi =="FC", "13:00",  np.where(dfi =="FD", "13:00",  
                        np.where(dfi =="ES", "13:00",np.where(dfi =="EI", "13:00",np.where(dfi =="EN", "10:00",np.where(dfi =="EA", "19:00",np.where(dfi =="EC", "13:00",  
                        np.where(dfi =="DA", "19:00",np.where(dfi =="EM", "19:00",np.where(dfi =="FQ", "19:00",np.where(dfi =="EX", "19:00",np.where(dfi =="ER", "19:00",np.where(dfi =="EO", "19:00",
                        np.where(dfi =="EO", "19:00",np.where(dfi =="EH", "19:00",np.where(dfi =="VO", "19:00",np.where(dfi =="DB", "19:00",np.where(dfi =="EQ", "19:00",np.where(dfi =="DW", "19:00",np.where(dfi =="PA", "19:00",np.where(dfi =="DT", "19:00",
                        np.where(dfi =="TG", "19:00",np.where(dfi =="RH", "19:00",np.where(dfi =="RC", "19:00",np.where(dfi =="RU", "19:00",np.where(dfi =="RG", "19:00",np.where(dfi =="RY", "19:00",np.where(dfi =="RF", "19:00",np.where(dfi =="RG", "19:00",
                        np.where(dfi =="AB", "19:00",np.where(dfi =="RL", "19:00",np.where(dfi =="OC", "19:00",np.where(dfi =="LH", "19:00",np.where(dfi =="RA", "19:00",np.where(dfi =="AD", "19:00",
                        np.where(dfi =="RE", "19:00",np.where(dfi =="RV", "19:00",np.where(dfi =="RM", "19:00",np.where(dfi =="CO", "19:00",np.where(dfi =="UG", "19:00",
                        np.where(dfi =="LS", "19:00",np.where(dfi =="LA", "19:00",np.where(dfi =="RR", "19:00",np.where(dfi =="RQ", "19:00",np.where(dfi =="LD", "19:00",
                        np.where(dfi =="QA", "19:00","19:00")))))))))))))))))))))))))))))))))))))))))))))))))                                                                        

        dfif2=pd.DataFrame(dfiaux2,columns=['Fim do Intervalo da Entrega'])                                                                                  
        dfif2=np.where(dfi3['JANELA_HORARIA'].str.len() >= 4, dfifaux,dfiaux2)
        dfif2=pd.DataFrame(dfif2,columns=['Fim do Intervalo da Entrega'])


        # dfif.to_excel("dfif.xlsx", index=False)

        # dfif2.to_excel("dfif2.xlsx", index=False)

        # print(df1)
        # print(df2)
        # print(df3)
        # print(df4)

        # print(df6)
        # print(df7)


        df10={
            'Capacidade 1':[1],'Capacidade 2':[1],'Capacidade 3':[1],'Capacidade 4':[''],'Capacidade 5':[''],
            'Capacidade 6':[''],'Capacidade 7':[''],'Capacidade 8':[''],'Capacidade 9':[''],'Capacidade 10':[''],
            'Tipo de Visita':['Entrega'],'Localização da recolha':[''],'Código Postal de Recolha':[''],'Cidade de Recolha':['']
            ,'País de Recolha':[''],'Latitude da recolha':[''],'Longitude da recolha':[''],'Inicio do Intervalo de Recolha':[''],'Fim do Intervalo de Recolha':['']}

        df10 = pd.DataFrame(df10,columns=['Capacidade 1','Capacidade 2','Capacidade 3','Capacidade 4','Capacidade 5','Capacidade 6','Capacidade 7',
                                        'Capacidade 8','Capacidade 9','Capacidade 10','Tipo de Visita','Localização da recolha','Código Postal de Recolha','Cidade de Recolha',
                                        'País de Recolha','Latitude da recolha','Longitude da recolha','Inicio do Intervalo de Recolha','Fim do Intervalo de Recolha'])



        df11={'Tempo de Carga':[''],'Tempo de Descarga':[''],'Tempo Máximo até à visita':[''],'Referência':[''],'Referência 1':[''],'Referência 2':[''],'Referência 3':['']}
            


        df14={'Prioridade':[''],'Benefício':[''],'Armazém':[''],'Permite transbordo':['Não'],'Tempo do Serviço Incluído na Janela Horária':['Não'],'Retorno 1':[''],'Retorno 2':[''],
        'Retorno 3':[''],'Retorno 4':[''],'Retorno 5':[''],'Retorno 6':[''],'Retorno 7':[''],'Retorno 8':[''],'Retorno 9':[''],'Retorno 10':[''],'Disponível para carga':[''],
        'Permitir Carga Completa':['Sim'],'Tempo de Estacionamento':[''],'Contrato':[''],'Tempo de estacionamento incluído na janela horária':['Sim'],'Cross-docking':[''],
        'Nº Portas':[''],'Dividir de':[''],'Dividir em':[''],'Tabela de Tempo de Visita':[''],'Produtos para Recolha':[''],'Produtos para Entrega':[''],'Data de Entrada no Sistema':[''],
        'Classificação de prioridade':[''],'Referências de entidade':[''],'Qualquer Armazém':[''],'Chave identificadora':[''],'É Direta':['Não'],'Minutos de desvio':[''],
        'Início da volta após os minutos de desvio':['']}
                

        df11 = pd.DataFrame(df11,columns=['Tempo de Carga','Tempo de Descarga','Tempo Máximo até à visita','Referência','Referência 1','Referência 2','Referência 3'])



        df14 =pd.DataFrame(df11,columns=['Prioridade','Benefício','Armazém','Permite transbordo','Tempo do Serviço Incluído na Janela Horária','Retorno 1','Retorno 2','Retorno 3','Retorno 4','Retorno 5','Retorno 6',
        'Retorno 7','Retorno 8','Retorno 9','Retorno 10','Disponível para carga','Permitir Carga Completa','Tempo de Estacionamento','Contrato','Tempo de estacionamento incluído na janela horária',
        'Cross-docking','Nº Portas','Dividir de','Dividir em','Tabela de Tempo de Visita','Produtos para Recolha','Produtos para Entrega','Data de Entrada no Sistema',
        'Classificação de prioridade','Referências de entidade','Qualquer Armazém','Chave identificadora','É Direta','Minutos de desvio','Início da volta após os minutos de desvio'])



        df5 = pd.DataFrame(df5,columns=['País de Entrega'])  
        df10_repeated = pd.concat([df10]*len(df), ignore_index=True)
        df11_repeated = pd.concat([df11]*len(df), ignore_index=True)
        df14_repeated = pd.concat([df14]*len(df), ignore_index=True)
        df5_repeated = pd.concat([df5]*len(df), ignore_index=True)

        # localização com janelas horárias reais
        
        df_l = pd.concat([df1, df10_repeated,df2_na,df3,df4,df5_repeated,df6,df7,dfif,dfif2,df11_repeated,df13,df14_repeated], axis=1)
        df_l = df_l.sort_values('Grupo', ascending=True)
        df_l = df_l.drop_duplicates(subset=['Nome'])
        df_l = df_l[~(df_l['Latitude da entrega'] == 0.0)]
        row_count0 = df_l.shape[0]
        
        df_l_otm = pd.concat([df1, df10_repeated,df2_na,df3,df4,df5_repeated,df6,df7,dfif,dfif2,df11_repeated,df13_vf,df14_repeated], axis=1)
        df_l_otm = df_l_otm.sort_values('Grupo', ascending=True)
        df_l_otm = df_l_otm.dropna(subset=['Grupo']).drop_duplicates(subset=['Nome'])
        df_l_otm = df_l_otm[~(df_l_otm['Latitude da entrega'] == 0.0)]
        df_l_otm = df_l_otm.dropna(subset=['Latitude da entrega'])
        df_l_otm = df_l_otm.dropna(subset=['Longitude da entrega'])
        df_l_otm = df_l_otm[df_l_otm['Latitude da entrega'] != 'nan']
        df_l_otm = df_l_otm.dropna(subset=['Longitude da entrega'])
        row_count = df_l_otm.shape[0]

        #-- localização com horário da entrega
        
        dfjhi = df13.copy()
        dfjhf = df13.copy()
        new_column_names_i = ['Inicio do Intervalo da Entrega']  # New column names
        new_column_names_f = ['Fim do Intervalo da Entrega']
        dfjhi.columns = new_column_names_i
        dfjhf.columns = new_column_names_f
        dfjhi=dfjhi['Inicio do Intervalo da Entrega'].str[:5]
        dfjhf = dfjhf['Fim do Intervalo da Entrega'].str[:5]

        df_l_h = pd.concat([df1, df10_repeated,df2_na,df3,df4,df5_repeated,df6,df7,dfjhi,dfjhf,df11_repeated,df13,df14_repeated], axis=1)
        df_l_h = df_l_h.sort_values('Grupo', ascending=True)
        df_l_h = df_l_h.drop_duplicates(subset=['Nome'])
        df_l_h['Localização da entrega'] = df_l_h['Localização da entrega'].str.replace(',', ' ')
        df_l_h['Localização da entrega'] = df_l_h['Localização da entrega'].str.replace('-', ' ')
        df_l_h['Localização da entrega'] = df_l_h['Localização da entrega'].apply(lambda x: 'x' if len(x.strip()) < 1 else x)
        df_l_h = df_l_h[df_l_h['Latitude da entrega'] != 'nan']


        # df_l_h = df_l_h[~(df_l_h['Latitude da entrega'] == 0.0)]
        # df_l_h = df_l_h[~(df_l_h['Inicio do Intervalo da Entrega'] == '00:00')]
        # df_l_h = df_l_h[~(df_l_h['Inicio do Intervalo da Entrega'].str[:2] == '00')]
        # df_l_h = df_l_h[~(df_l_h['Inicio do Intervalo da Entrega'].str[:2] == '01')]
        # df_l_h = df_l_h[~(df_l_h['Inicio do Intervalo da Entrega'].str[:2] == '02')]
        # df_l_h = df_l_h[~(df_l_h['Inicio do Intervalo da Entrega'].str[:2] == '03')]
        # df_l_h = df_l_h[~(df_l_h['Inicio do Intervalo da Entrega'].str[:2] == '04')]
        # df_l_h = df_l_h[~(df_l_h['Inicio do Intervalo da Entrega'].str[:2] == '05')]
        # df_l_h = df_l_h[~(df_l_h['Inicio do Intervalo da Entrega'].str[:2] == '06')]
        # df_l_h = df_l_h[~(df_l_h['Inicio do Intervalo da Entrega'].str[:2] == '23')]

        row_count0 = df_l_h.shape[0]
        
        # Define the OneDrive folder path
        # Define the OneDrive folder path
        if os.name == 'nt':  # Windows
            one_drive_folder = r'C:\Users\1420844\OneDrive - CTT - Correios de Portugal\01. Last-mile\06. Otimização Last Mile Expresso\Python\03.Modelo_parcial'
        else:  # Assuming Unix-like (Linux, macOS)
            one_drive_folder = "/Users/rubenfilipemartinsdarocha/Library/CloudStorage/GoogleDrive-rubenfilype@gmail.com/My Drive/Colab Notebooks/CTT/03.Modelo_parcial"
        # Construct the full file path
        file_name = str(CO) + '.xlsx'  # Replace CO with the appropriate variable name
        file_path = os.path.join(one_drive_folder, file_name)

        # Read Excel sheets
        df_v = pd.read_excel(file_path, sheet_name='Veículos')
        df_r = pd.read_excel(file_path, sheet_name='Regras')
        df_s = pd.read_excel(file_path, sheet_name='Semi-Reboques')


        #-

        #df_seq_rota = pd.DataFrame(pd.read_csv(open(file_name+".csv"),sep=";",encoding='utf-8', na_values=['\n']))
        header = {'Nome;':[],'ID Grupo Fundido;':[], 'Prioridade na Sequência de Consolidações;':[], 'Sequência de Desconsolidações;':[],
                'Sequência de doca;':[], 'Chave Semi-Reboque;':[], 'Sequência de Semi-Reboque;':[], 'Turno;':[]}

        header_2 = pd.DataFrame(header,columns=['Nome','ID Grupo Fundido', 'Prioridade na Sequência de Consolidações', 'Sequência de Desconsolidações',
                'Sequência de doca', 'Chave Semi-Reboque', 'Sequência de Semi-Reboque', 'Turno'])

        df_seq_rota=pd.concat([header_2, df_seq_rota], axis=0)


        

        downloads_folder = os.path.expanduser('~/Downloads')


        # Assuming you have defined your DataFrames (df_v, df_l, df_r, df_s, df_seq_rota)

        # Construct the full file path
        file_name = str(CO) +'_'+str(date_str)+'_R_' + str(row_count0)
        row_count0 = 0  # Replace with the appropriate value

        full_file_path = os.path.join(downloads_folder, f"{file_name}.xlsx")
        
        #--janelas horárias reais -- 

        #with pd.ExcelWriter(file_name+'_R_'+str(row_count0)+'.xlsx') as writer:
        #    df_v.to_excel(writer,sheet_name='Veículos', index=False)
        #    df_l.to_excel(writer,sheet_name='Localizações', index=False)
        #    df_r.to_excel(writer,sheet_name='Regras', index=False)
        #    df_s.to_excel(writer,sheet_name='Semi-Reboques', index=False)
        #    df_seq_rota.to_excel(writer,sheet_name='Rotas', index=False)
            
        # with pd.ExcelWriter(full_file_path) as writer:
        #     df_l.to_excel(writer,sheet_name='Localizações', index=False)
            
        #with pd.ExcelWriter(file_name+'_L_otm_'+str(row_count)+'.xlsx') as writer:
        #    df_l_otm.to_excel(writer,sheet_name='Localizações', index=False)

        # janelas horárias com hora real de entrega
        #with pd.ExcelWriter(file_name+'_R_'+str(row_count0)+'.xlsx') as writer:
        #    df_v.to_excel(writer,sheet_name='Veículos', index=False)
        #    df_l_h.to_excel(writer,sheet_name='Localizações', index=False)
        #    df_r.to_excel(writer,sheet_name='Regras', index=False)
        #    df_s.to_excel(writer,sheet_name='Semi-Reboques', index=False)
        #    df_seq_rota.to_excel(writer,sheet_name='Rotas', index=False)
        
    # Assuming you have defined your DataFrames (df_v, df_l, df_r, df_s, df_seq_rota)

    #Construct the full file path
        # file_name = file_name +str(date_str)+'_R_' + str(row_count0)
        # row_count0 = 0  # Replace with the appropriate value
        # full_file_path = os.path.join(downloads_folder, f"{file_name}.xlsx")

        #Save the Excel file
        
        with pd.ExcelWriter(full_file_path, engine='xlsxwriter') as writer:
            df_v.to_excel(writer,sheet_name='Veículos', index=False)
            df_l_h.to_excel(writer,sheet_name='Localizações', index=False)
            df_r.to_excel(writer,sheet_name='Regras', index=False)
            df_s.to_excel(writer,sheet_name='Semi-Reboques', index=False)
            df_seq_rota.to_excel(writer,sheet_name='Rotas', index=False)
            
        print(file_name)
  


8818171_16112023_R_4932
8818171_15112023_R_4906
8818171_17112023_R_4545
8818171_18112023_R_1431
8818171_20112023_R_5630
8818171_21112023_R_4519
8818171_22112023_R_5193
8818171_24112023_R_5131
8818171_23112023_R_5125
8818171_27112023_R_5281
8818171_25112023_R_1829
8818171_29112023_R_5127
8818171_28112023_R_5690
8818171_30112023_R_4968
8818171_13112023_R_4329
8818171_14112023_R_5126
